# Multi-effect distillation (MED) model validation
Alternative using Keras instead of MATLAB's ANNs toolbox

In [1]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
import time
from loguru import logger
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot
import re
import sys
import tensorflow as tf

%load_ext autoreload

data_path: Path = Path("../../data")

with open(data_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)
    
with open(data_path / 'plt_config_med.json') as f:
    plt_config = json.load(f)


2024-09-21 22:00:44.041031: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-21 22:00:44.044106: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-21 22:00:44.051216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 22:00:44.061449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 22:00:44.065242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 22:00:44.075842: I tensorflow/core/platform/cpu_feature_gu

In [4]:
# Load trained model
model = tf.keras.models.load_model(data_path/'med_ann_model.keras')

# Load dataset just to get mean and std, in order to normalize the data
# This should then be set as a constant in the SolarMED class to avoid having to load the dataset every time
med_dataset: pd.DataFrame = pd.read_csv(data_path / 'datasets/operation_points_v2.csv')
dataset_mean = med_dataset.mean()
dataset_std = med_dataset.std()

# Load experimental data
df = pd.read_csv(data_path / 'dev.csv', parse_dates=['time'], index_col='time')

2024-09-21 22:02:01.963730: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential_11', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 5], 'dtype': 'float64', 'sparse': False, 'ragged': False, 'name': 'normalization_11_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Normalization', 'config': {'name': 'normalization_11', 'trainable': True, 'dtype': 'float32', 'axis': [-1], 'invert': False, 'mean': None, 'variance': None}, 'registered_name': None, 'build_config': {'input_shape': [None, 5]}}, {'module': 'keras.layers', 'class_name': 'Dense', 'config': {'name': 'dense_121', 'trainable': True, 'dtype': 'float32', 'batch_input_shap

In [ ]:
idx_start = 500
span = 70
idx_end = len(df)
df_mod = pd.DataFrame()

outputs_norm = model.predict()
# Denormalize outputs
# outputs = outputs_norm * dataset_std + dataset_mean
outputs = (outputs_norm * np.array(dataset_std).reshape(1, -1)) + \
            np.full(outputs_norm.shape, np.array(dataset_mean).reshape(1, -1))

## Visualize results

And compare with experimental data

In [ ]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

%autoreload 2

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'med_keras_validation_{df.index[0].strftime("%Y%m%d")}')
)